<a href="https://www.kaggle.com/code/tusharhatwar785/pyspark-imdb-analysis?scriptVersionId=125251578" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 17.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=ff04f2b33cb0fdd0072eda76b36d10abbea1f86b8bf6e0449c307825f0abbe4a
  Stored in directory: /root/.cache/pip/wheels/5a/54/9b/a89cac960efb57c4c35d41cc7c9f7b80daa21108bc376339b7
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType
from pyspark.sql.functions import split, count, when, isnan, col, regexp_replace
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/d/PromptCloudHQ/imdb-data/IMDB-Movie-Data.csv


In [3]:
spark = SparkSession.builder.appName('First Session').getOrCreate()

print('Spark Version: {}'.format(spark.version))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/10 11:18:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Spark Version: 3.3.2


In [4]:
#Defining a Schema
csvSchema = StructType([StructField('rank', IntegerType(), nullable = True),
                     StructField('title', StringType(), nullable = True),
                     StructField('genre', StringType(), nullable = True),
                     StructField('description', StringType(), nullable = True),
                     StructField('director', StringType(), nullable = True),
                     StructField('actors', StringType(), nullable = True),
                     StructField('year', IntegerType(), nullable = True),
                     StructField('Runtime (Minutes)', IntegerType(), nullable = True),
                     StructField('rating', FloatType(), nullable = True),
                     StructField('votes', IntegerType(), nullable = True),
                     StructField('Revenue (Millions)', FloatType(), nullable = True),
                     StructField('metascore', FloatType(), nullable = True)
                    ])

file_path = '/kaggle/input/d/PromptCloudHQ/imdb-data/IMDB-Movie-Data.csv'


df = spark.read.csv(file_path,
                    header = True,
                    schema = csvSchema,
                    nanValue = '?')
df.show(5)

+----+--------------------+--------------------+--------------------+--------------------+--------------------+----+-----------------+------+------+------------------+---------+
|rank|               title|               genre|         description|            director|              actors|year|Runtime (Minutes)|rating| votes|Revenue (Millions)|metascore|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+----+-----------------+------+------+------------------+---------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|2014|              121|   8.1|757074|            333.13|     76.0|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|2012|              124|   7.0|485820|            126.46|     65.0|
|   3|               Split|     Horror,Thriller|Three girls are k...|  M. Night Shyamalan|James McAvoy, Any...

In [5]:
#Check Missing Values
def check_missing(dataframe):
    
    return dataframe.select([count(when(isnan(c) | col(c).isNull(), c)). \
                             alias(c) for c in dataframe.columns]).show()

check_missing(df)

+----+-----+-----+-----------+--------+------+----+-----------------+------+-----+------------------+---------+
|rank|title|genre|description|director|actors|year|Runtime (Minutes)|rating|votes|Revenue (Millions)|metascore|
+----+-----+-----+-----------+--------+------+----+-----------------+------+-----+------------------+---------+
|   0|    0|    0|          0|       0|     0|  12|                5|     1|    7|               124|       68|
+----+-----+-----+-----------+--------+------+----+-----------------+------+-----+------------------+---------+



In [6]:
df.printSchema()

root
 |-- rank: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- description: string (nullable = true)
 |-- director: string (nullable = true)
 |-- actors: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- Runtime (Minutes): integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- votes: integer (nullable = true)
 |-- Revenue (Millions): float (nullable = true)
 |-- metascore: float (nullable = true)



In [7]:
#Handling Missing Values
df = df.na.drop()

df = df.withColumn("metascore", df["metascore"].cast(IntegerType()))
#convert votes from string to int

df.show(5)

+----+--------------------+--------------------+--------------------+--------------------+--------------------+----+-----------------+------+------+------------------+---------+
|rank|               title|               genre|         description|            director|              actors|year|Runtime (Minutes)|rating| votes|Revenue (Millions)|metascore|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+----+-----------------+------+------+------------------+---------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|2014|              121|   8.1|757074|            333.13|       76|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|2012|              124|   7.0|485820|            126.46|       65|
|   3|               Split|     Horror,Thriller|Three girls are k...|  M. Night Shyamalan|James McAvoy, Any...

In [8]:
df.printSchema()

root
 |-- rank: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- description: string (nullable = true)
 |-- director: string (nullable = true)
 |-- actors: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- Runtime (Minutes): integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- votes: integer (nullable = true)
 |-- Revenue (Millions): float (nullable = true)
 |-- metascore: integer (nullable = true)



In [9]:
#Check column names
df.columns

['rank',
 'title',
 'genre',
 'description',
 'director',
 'actors',
 'year',
 'Runtime (Minutes)',
 'rating',
 'votes',
 'Revenue (Millions)',
 'metascore']

In [10]:
#Renaming Columns
df = df.withColumnRenamed('REVENUE', 'revenue') \
.withColumnRenamed('Runtime (Minutes)', 'runtime')


In [11]:
#Check column names
df.columns

['rank',
 'title',
 'genre',
 'description',
 'director',
 'actors',
 'year',
 'runtime',
 'rating',
 'votes',
 'Revenue (Millions)',
 'metascore']

In [12]:
#Display data with pandas format
df.toPandas().head()

,rank,title,genre,description,director,actors,year,runtime,rating,votes,Revenue (Millions),metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.130005,76
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.459999,65
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.119995,62
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.320007,59
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.019989,40


In [13]:
#Check the schema
df.printSchema()

root
 |-- rank: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- description: string (nullable = true)
 |-- director: string (nullable = true)
 |-- actors: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- votes: integer (nullable = true)
 |-- Revenue (Millions): float (nullable = true)
 |-- metascore: integer (nullable = true)



In [14]:
#get first 3 rows
df.show(3)

+----+--------------------+--------------------+--------------------+------------------+--------------------+----+-------+------+------+------------------+---------+
|rank|               title|               genre|         description|          director|              actors|year|runtime|rating| votes|Revenue (Millions)|metascore|
+----+--------------------+--------------------+--------------------+------------------+--------------------+----+-------+------+------+------------------+---------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|        James Gunn|Chris Pratt, Vin ...|2014|    121|   8.1|757074|            333.13|       76|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|      Ridley Scott|Noomi Rapace, Log...|2012|    124|   7.0|485820|            126.46|       65|
|   3|               Split|     Horror,Thriller|Three girls are k...|M. Night Shyamalan|James McAvoy, Any...|2016|    117|   7.3|157606|            138.12|       62|
+---

In [15]:
#describe with specific variables
df.describe(['title', 'year', 'runtime']).show()

+-------+--------------------+------------------+------------------+
|summary|               title|              year|           runtime|
+-------+--------------------+------------------+------------------+
|  count|                 830|               830|               830|
|   mean|   635.6666666666666|2012.4927710843374|114.68192771084337|
| stddev|   860.1559548515994|3.1694136528550287|18.463932464931663|
|    min|(500) Days of Summer|              2006|                66|
|    max|            Zootopia|              2016|               187|
+-------+--------------------+------------------+------------------+



In [16]:
#describe with numerical columns
def get_num_cols(dataframe):
    
    num_cols = [col for col in dataframe.columns if dataframe.select(col). \
                dtypes[0][1] in ['double', 'int']]
    
    return num_cols

num_cols = get_num_cols(df)
    
df.describe(num_cols).show()

+-------+-----------------+------------------+------------------+------------------+------------------+
|summary|             rank|              year|           runtime|             votes|         metascore|
+-------+-----------------+------------------+------------------+------------------+------------------+
|  count|              830|               830|               830|               830|               830|
|   mean|484.8975903614458|2012.4927710843374|114.68192771084337| 194130.4313253012| 59.59277108433735|
| stddev| 286.927087519967|3.1694136528550287|18.463932464931663|193659.28426879045|16.966736655765217|
|    min|                1|              2006|                66|               178|                11|
|    max|             1000|              2016|               187|           1791916|               100|
+-------+-----------------+------------------+------------------+------------------+------------------+



In [17]:
#Get movies with rating > 7 
df.filter(df['rating'] > 7).show(5)

+----+--------------------+--------------------+--------------------+--------------------+--------------------+----+-------+------+------+------------------+---------+
|rank|               title|               genre|         description|            director|              actors|year|runtime|rating| votes|Revenue (Millions)|metascore|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+----+-------+------+------+------------------+---------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|2014|    121|   8.1|757074|            333.13|       76|
|   3|               Split|     Horror,Thriller|Three girls are k...|  M. Night Shyamalan|James McAvoy, Any...|2016|    117|   7.3|157606|            138.12|       62|
|   4|                Sing|Animation,Comedy,...|In a city of huma...|Christophe Lourdelet|Matthew McConaugh...|2016|    108|   7.2| 60545|            270.32|   

In [18]:
#Multiple Conditions
#Select movie with runtime greater than 120, year greater than 2015 and get its title
df.filter((df['runtime'] > 120) & 
          (df['year'] > 2015)).select('title').show(5)

+--------------------+
|               title|
+--------------------+
|       Suicide Squad|
|          La La Land|
|  The Lost City of Z|
|Fantastic Beasts ...|
|      Hidden Figures|
+--------------------+
only showing top 5 rows



In [19]:
#Sorting
#movies which were released after 2015 and with rating from high to low
df.filter((df['year'] > 2015)).orderBy('rating', ascending = False).show(5)

+----+-------------+--------------------+--------------------+---------------+--------------------+----+-------+------+------+------------------+---------+
|rank|        title|               genre|         description|       director|              actors|year|runtime|rating| votes|Revenue (Millions)|metascore|
+----+-------------+--------------------+--------------------+---------------+--------------------+----+-------+------+------+------------------+---------+
|  97|Kimi no na wa|Animation,Drama,F...|Two strangers fin...| Makoto Shinkai|Ryûnosuke Kamiki,...|2016|    106|   8.6| 34110|              4.68|       79|
|   7|   La La Land|  Comedy,Drama,Music|A jazz pianist fa...|Damien Chazelle|Ryan Gosling, Emm...|2016|    128|   8.3|258682|            151.06|       93|
|  17|Hacksaw Ridge|Biography,Drama,H...|WWII American Arm...|     Mel Gibson|Andrew Garfield, ...|2016|    139|   8.2|211760|             67.12|       71|
|  19|         Lion|     Biography,Drama|A five-year-old I...|  

## Filtering with SQL

In [20]:
df.select("director").show()

+--------------------+
|            director|
+--------------------+
|          James Gunn|
|        Ridley Scott|
|  M. Night Shyamalan|
|Christophe Lourdelet|
|          David Ayer|
|         Yimou Zhang|
|     Damien Chazelle|
|          James Gray|
|       Morten Tyldum|
|         David Yates|
|      Theodore Melfi|
|      Gareth Edwards|
|        Ron Clements|
|     Nacho Vigalondo|
|        Chris Renaud|
|          Mel Gibson|
|     Paul Greengrass|
|         Garth Davis|
|    Denis Villeneuve|
|      Stephen Gaghan|
+--------------------+
only showing top 20 rows



In [21]:
#Get the director with 'ar' in their names
df.filter("director like '%ar%'").select("director").show(10)

+--------------------+
|            director|
+--------------------+
|      Gareth Edwards|
|         Garth Davis|
|       Barry Jenkins|
|Ricardo de Montreuil|
|        Rob Marshall|
|      Garry Marshall|
|     Martin Scorsese|
|        Fede Alvarez|
|       Thea Sharrock|
|        Byron Howard|
+--------------------+
only showing top 10 rows



## Group by Operations

In [22]:
# total number of Movies directed by a director
df.groupBy('director').count().alias("sum_salary").orderBy('count', ascending = False).show(5)

+------------------+-----+
|          director|count|
+------------------+-----+
|      Ridley Scott|    8|
|Paul W.S. Anderson|    6|
|       David Yates|    6|
|       Michael Bay|    6|
|       Zack Snyder|    5|
+------------------+-----+
only showing top 5 rows



In [23]:
#show total movies directed by a director in Descending order
df.groupBy('director') \
.count() \
.withColumnRenamed("count", "movies_directed") \
.orderBy('count', ascending = False) \
.show(5)

+------------------+---------------+
|          director|movies_directed|
+------------------+---------------+
|      Ridley Scott|              8|
|Paul W.S. Anderson|              6|
|       David Yates|              6|
|       Michael Bay|              6|
|       Zack Snyder|              5|
+------------------+---------------+
only showing top 5 rows

